# importing library

In [2]:
from __future__ import print_function
import keras
from keras.models import Model,load_model
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import plot_model
from PIL import Image , ImageOps
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.resnet50 import ResNet50
import random
import os
import math
import tensorflow as tf
import numpy as np
import densenet
import os
from shutil import copyfile
from tensorflow.python.framework import graph_util

Using TensorFlow backend.


In [14]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [13]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1918658016147652860
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 582994049396728377
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 18185636121376598658
physical_device_desc: "device: XLA_GPU device"
]


# Creating train and test set

In [4]:
# Creation of Test data
input_folder = './DataSet'

x_train =[]
y_train =[]
x_test  =[]
y_test  =[]

percent = 0.7

for classes in os.listdir(input_folder):
    print(classes,' ',end = '')
    imagePath = input_folder+'/'+classes
    class_data = []
    class_data_name = []
    for images in os.listdir(imagePath):
        try:
            im = Image.open(imagePath+'/'+images)
            imgMat = np.array(im)
            if imgMat.shape == (224,224,3): 
                class_data.append(imgMat)
                class_data_name.append(imagePath+'/'+images)
        except:
            print('LoL')
#     print(len(class_data))
    sizeofclass = len(class_data)
    
    num_of_Train_exp = math.floor(sizeofclass*(percent))
    
    X_train = class_data[0:num_of_Train_exp]
    X_test = class_data[num_of_Train_exp:]
    
    TrainSet = class_data_name[0 : num_of_Train_exp]
    ValidationSet = class_data_name[num_of_Train_exp : ]

    directory = './DATASET/Training/'+classes
    if not os.path.exists(directory):
        os.makedirs(directory)
    i = 1
    for imagesPath in TrainSet:
        dest2 = directory +'/'+str(i)+'.jpg'
        copyfile(imagesPath, dest2)
        i = i + 1
    
    
    directory = './DATASET/Validation/'+classes
    if not os.path.exists(directory):
        os.makedirs(directory)
    i = 1
    for imagesPath in ValidationSet:
        dest2 = directory +'/'+str(i)+'.jpg'
        copyfile(imagesPath, dest2)
        i = i + 1

#     Y_train = [int(classes)-1]*len(X_train)
#     Y_test = [int(classes)-1]*len(X_test)
    
#     x_train = x_train + X_train
#     y_train = y_train + Y_train
#     x_test = x_test + X_test
#     y_test = y_test + Y_test

# del X_train
# del Y_train
# del X_test
# del Y_test

3  8  60  11  44  20  17  47  10  36  52  30  5  49  12  50  61  2  46  19  LoL
9  25  48  62  55  LoL
29  14  18  21  26  40  33  15  22  27  53  13  4  63  16  28  32  39  35  37  6  56  54  1  7  41  58  23  64  43  31  45  65  38  34  LoL
59  42  57  24  51  

# CNN Architecture

# Model 1

In [2]:
# input image dimensions
# img_rows, img_cols ,channel = 224, 224 , 3
def createModel1():
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3,3),strides=1,padding='same',activation='relu',input_shape=(224,224,3)))
    model.add(Conv2D(filters=20, kernel_size=(3,3),strides=1,activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=9, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=5, padding="same"))
    model.add(Flatten())
    model.add(Dense(1089,activation='relu'))
    model.add(Dense(400,activation='relu'))
    model.add(Dense(20,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])    
    return model

# Model 2

In [15]:
def createModel2():
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model.output

    x = GlobalAveragePooling2D()(x)
    x = Dense(400, activation='relu')(x)
    
    predictions = Dense(20, activation='softmax')(x)

    m = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    for layer in m.layers[:249]:
        layer.trainable = False
    for layer in m.layers[249:]:
        layer.trainable = True

    m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return m

# Model 3

In [21]:
def createModel3():
    base_model = VGG16(weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(400, activation='relu')(x)
    # and a logistic layer -- 
    predictions = Dense(20, activation='softmax')(x)

    m = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    for layer in m.layers[:249]:
        layer.trainable = False
    for layer in m.layers[249:]:
        layer.trainable = True

    m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return m

# Model4

In [29]:
# input image dimensions
img_rows, img_cols ,channel = 224, 224 , 3


# convert class vectors to binary class matrices
def createModel4():
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3,3),strides=1,padding='same',activation='relu',input_shape=(224,224,3)))
    model.add(Conv2D(filters=1096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=2096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=2096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=3096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=3096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=4096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=5012, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=5012, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=6012, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=7017, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    
    model.add(Conv2D(filters=256, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=512, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=512, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=1096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=1096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=2096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=3096, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=3096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=4096, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    #model.add(Conv2D(filters=128, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    
    model.add(Flatten())
    model.add(Dense(5096,activation='relu'))
    model.add(Dense(2000,activation='relu'))
    model.add(Dense(20,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
    model.summary()
    return model

# Model 

In [19]:
# input image dimensions
img_rows, img_cols ,channel = 224, 224 , 3


# convert class vectors to binary class matrices
def createMode10():
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3,3),strides=1,padding='same',activation='relu',input_shape=(224,224,3)))
    model.add(Conv2D(filters=32, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=256, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Conv2D(filters=128, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(1,1),strides=1,padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding="same"))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dense(2000,activation='relu'))
    model.add(Dense(20,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
    print(model.summary())
    return model

# Model 5

In [9]:
def createModel5():
    base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(224,224,3))

    # # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(400, activation='relu')(x)
    # and a logistic layer -- 
    predictions = Dense(20, activation='softmax')(x)

    m = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    for layer in m.layers[-3:]:
        layer.trainable = True

    m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return m

# Model 6

In [2]:
def createModel6():
    #input_format = Input(shape=(224,224,3),name = 'image_input')
    #base_model = MobileNetV2(weights='imagenet', include_top=False,input_tensor=input_format)
    base_model= keras.applications.densenet.DenseNet121(include_top=False,input_shape=(224,224,3))
    # add a global spatial average pooling layer
        
    x = base_model.output
    
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(400, activation='relu')(x)
    # and a logistic layer -- 
    predictions = Dense(65, activation='softmax',name='output')(x)

    m = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = True

    for layer in m.layers:
        layer.trainable = True
        
    for layer in m.layers[-2:]:
        layer.trainable = True


    m.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return m

# Generator

In [32]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=False,
        featurewise_std_normalization=False,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './DATASET/Training',
        target_size=(224, 224),
        batch_size=10,
        save_to_dir='./Augmented/Training',
        save_prefix='aug', 
        save_format='jpg',
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        './DATASET/Validation',
        target_size=(224, 224),
        batch_size=10,
        save_to_dir='./Augmented/Validation',
        save_prefix='aug', 
        save_format='jpg',
        class_mode='categorical',
        shuffle=False
    )


# i = 0
# for batch in train_generator:
#     i += 1
#     if i > 20:
#         break


Found 14737 images belonging to 65 classes.
Found 6344 images belonging to 65 classes.


Found 6344 images belonging to 65 classes.


In [41]:
generator = test_datagen.flow_from_directory(
        './',
        target_size=(224, 224),
        batch_size=10,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict_generator(generator, 65)

Found 262161 images belonging to 11 classes.


/home/kuldeep/anaconda3/envs/AdvanceCV/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


OSError: broken data stream when reading image file

# Training and Testing

In [13]:
# config
tf.reset_default_graph()
K.clear_session()
epochs = 10

checkpoint_path = "training_4/Checkpoints/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1,period=1)

# model = createModel6()
# print(model.summary())


image_dim = (224, 224, 3)

model = densenet.DenseNet(classes=65, input_shape=image_dim, depth=19, growth_rate=12,bottleneck=True, reduction=0.5)
print(model.summary())
model.compile(loss="categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])
model.fit_generator(
        train_generator,
#         steps_per_epoch=100,
        callbacks=[cp_callback],
        epochs=epochs,
        validation_data=validation_generator,
#         validation_steps=200
        
)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 24) 648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 224, 224, 24) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 224, 224, 24) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

Epoch 1/10
1474/1474 [==============================] - 10045s 7s/step - loss: 3.8333 - acc: 0.1229 - val_loss: 3.6801 - val_acc: 0.1472

Epoch 00001: saving model to training_4/Checkpoints/cp-0001.ckpt
Epoch 2/10
1474/1474 [==============================] - 10007s 7s/step - loss: 3.5539 - acc: 0.1562 - val_loss: 3.6972 - val_acc: 0.1561

Epoch 00002: saving model to training_4/Checkpoints/cp-0002.ckpt
Epoch 3/10
1474/1474 [==============================] - 9896s 7s/step - loss: 3.4094 - acc: 0.1810 - val_loss: 3.7558 - val_acc: 0.1557

Epoch 00003: saving model to training_4/Checkpoints/cp-0003.ckpt
Epoch 4/10
1474/1474 [==============================] - 9910s 7s/step - loss: 3.3004 - acc: 0.1956 - val_loss: 3.2008 - val_acc: 0.2133

Epoch 00004: saving model to training_4/Checkpoints/cp-0004.ckpt
Epoch 5/10
1474/1474 [==============================] - 9979s 7s/step - loss: 3.2259 - acc: 0.2066 - val_loss: 3.4081 - val_acc: 0.1734

Epoch 00005: saving model to training_4/Checkpoints/c

In [4]:
model = densenet.DenseNet(classes=65, input_shape=(224,224,3), depth=19, growth_rate=12,bottleneck=True, reduction=0.5)
model.load_weights('training_4/Checkpoints/cp-0010.ckpt')

In [5]:
model.save('newModel.h5')

In [ ]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd)

In [16]:
print(len(validation_generator))

635


In [17]:
#score = model.evaluate(train_generator, validation_data)

model.compile(loss="categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])
score= model.evaluate_generator(validation_generator, max_queue_size=10)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.9698621852025107
Test accuracy: 0.27411728183425316


In [44]:
model.history['acc']

AttributeError: 'Model' object has no attribute 'history'

In [41]:
predicted= model.predict_generator(validation_generator)

In [42]:
#predicted[0]
#y_pred = np.rint(predicted)
#y_true = validation_generator.classes
#print (classification_report(y_true, y_pred))

In [43]:
from sklearn.metrics import classification_report 
from keras.utils import np_utils
y_curr = np.zeros(6344)
for i in range(0,6344):
    index = np.argmax(predicted[i])
    y_curr[i] = index

y_test= validation_generator.classes

cc = ['Bhindi','Briyani','Naan','Halwa', 'Imarti', 'Chole bhatura', 'Daal makhni', 'jalebi', 'Cachuri', 'Kadi', 'Barfi', 'kheer','kulfi', 'ladoo', 'Pav bhaaji', 'Momos', 'Papad', 'Ras malai', 'Roti', 'Upma', 'Aloo_gobi', 'Aloo_matar', 'Aloo_methi', 'Bajji', 'Bhatura', 'Bonda', 'Butter_Chicken', 'Chaap', 'Chana_masala', 'chicken_65', 'chole_kulche', 'Dahi_bhalla', 'Dahi_Vada', 'dal', 'Dhokla', 'Dosa', 'iddli', 'Kadai_paneer', 'Kabab', 'Khandvi', 'litti_chokha', 'Maggie', 'Malai_kofta', 'Matar_paneer', 'Modak', 'Palak_Paneer', 'Panipuri', 'Paratha', 'Poha', 'Puttu', 'RajmaChawal', 'Rajma', 'Rasam_Rice', 'Rice', 'RoganJosh', 'Sambar', 'Samosa', 'Shahi_paneer', 'Shrikhand', 'Springroll', 'Tandoori_Chicken', 'Thepla', 'Uttapam', 'Vada_pav', 'Vada' ]                                           
print(classification_report(y_test, y_curr,target_names=cc))

                  precision    recall  f1-score   support

          Bhindi       0.37      0.48      0.42        54
         Briyani       0.30      0.22      0.25        97
            Naan       0.27      0.60      0.38       140
           Halwa       0.23      0.82      0.36       523
          Imarti       0.44      0.06      0.11        62
   Chole bhatura       0.29      0.22      0.25       171
     Daal makhni       0.00      0.00      0.00        75
          jalebi       0.00      0.00      0.00        84
         Cachuri       0.56      0.08      0.14        63
            Kadi       0.41      0.17      0.24       215
           Barfi       0.75      0.04      0.07        82
           kheer       0.36      0.53      0.43       172
           kulfi       0.33      0.05      0.09       113
           ladoo       0.00      0.00      0.00        69
      Pav bhaaji       0.14      0.03      0.05        61
           Momos       0.31      0.06      0.10        65
           Pa

In [26]:
from sklearn.metrics import classification_report
y_pred = model.predict(train_generator)

AttributeError: 'DirectoryIterator' object has no attribute 'ndim'

In [ ]:
y_curr = np.zeros(y_pred.shape)
for i in range(0,len(y_pred)):
    index = np.argmax(y_pred[i])
    y_curr[i][index] = 1

target_names = ['Bhindi','Briyani','Naan','Halwa', 'Imarti', 'Chole bhatura', 'Daal makhni', 'jalebi', 'Cachuri', 'Kadi', 'Barfi', 'kheer','kulfi', 'ladoo', 'Pav bhaaji', 'Momos', 'Papad', 'Ras malai', 'Roti', 'Upma', 'Aloo_gobi', 'Aloo_matar', 'Aloo_methi', 'Bajji', 'Bhatura', 'Bonda', 'Butter_Chicken', 'Chaap', 'Chana_masala', 'chicken_65', 'chole_kulche', 'Dahi_bhalla', 'Dahi_Vada', 'dal', 'Dhokla', 'Dosa', 'iddli', 'Kadai_paneer', 'Kabab', 'Khandvi', 'litti_chokha', 'Maggie', 'Malai_kofta', 'Matar_paneer', 'Modak', 'Palak_Paneer', 'Panipuri', 'Paratha', 'Poha', 'Puttu', 'RajmaChawal', 'Rajma', 'Rasam_Rice', 'Rice', 'RoganJosh', 'Sambar', 'Samosa', 'Shahi_paneer', 'Shrikhand', 'Springroll', 'Tandoori_Chicken', 'Thepla', 'Uttapam', 'Vada_pav', 'Vada' ]                                           
print(classification_report(y_test, y_curr,target_names=target_names))

In [32]:
model = load_model('my_modelF.h5')
#score = model.evaluate(X_test, Y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
print(model.history.keys())

AttributeError: 'Model' object has no attribute 'history'

In [11]:
model = load_model('saveModel.h5')
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,output_names, freeze_var_names)
        return frozen_graph
    
    
frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "./training", "my_model.pb", as_text=False)

INFO:tensorflow:Froze 1920 variables.
INFO:tensorflow:Converted 1920 variables to const ops.


'./training/my_model.pb'

In [49]:
from keras.models import load_model

model.save('my_modelFF.h5')

# Garbage Code

In [ ]:
# X_TEST_DATA = []
# Y_TEST_DATA = []

# input_folder_test = './ResizedImg/test'
# for classes in os.listdir(input_folder):
#     imagePath = input_folder+'/'+classes
#     for images in os.listdir(imagePath):
#         im = Image.open(imagePath+'/'+images)
#         imgMat = np.array(im)
#         X_TEST_DATA.append(imgMat)
#         Y_TEST_DATA.append(classes)

# X_TRAIN_DATA = []
# Y_TRAIN_DATA = []

# input_folder_test = './ResizedImg/train'
# for classes in os.listdir(input_folder):
#     imagePath = input_folder+'/'+classes
#     for images in os.listdir(imagePath):
#         im = Image.open(imagePath+'/'+images)
#         imgMat = np.array(im)
#         X_TRAIN_DATA.append(imgMat)
#         Y_TRAIN_DATA.append(classes)

# x_train_F = [item for sublist in x_train for item in sublist]
# y_train_F = [item for sublist in y_train for item in sublist]
# x_test_F  = [item for sublist in x_test for item in sublist]
# y_test_F  = [item for sublist in y_test for item in sublist]

# x_train_w = []
# y_train_w = []
# x_test_w = []
# y_test_w = []

# for img in x_train:
#     try:
#         if img.shape == (224,224,3):
#             x_train_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in y_train:
#     try:
#         if img.shape == (224,224,3):
#             y_train_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in x_test:
#     try:
#         if img.shape == (224,224,3):
#             x_test_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# for img in y_test:
#     try:
#         if img.shape == (224,224,3):
#             y_test_w.append(np.array(img))
#         else:
#             print('wrong',img.shape)
#     except:
#         print('LOL')

# x_train = np.array(x_train_w, dtype=np.float32)
# y_train = np.array(y_train_w, dtype=np.float32)
# x_test  = np.array(x_test_w, dtype=np.float32)
# y_test  = np.array(y_test_w, dtype=np.float32)

# img = Image.fromarray(np.array(X_train[0]))
# img


# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))




# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train[0])
# print(len(x_train[0]))
# im = Image.open(x_train[0])
# im.show()
# print('before changing')
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')


# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
#     print("hola")
# else:
#     print("hola8")
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# print(x_train[0])
# print(len(x_train[0][0][0]))
# print(x_train[0].shape)
# print(np.zeros((3,3,1)))

# im1 = Image.open("b.jpeg").convert('LA')
# im1 = Image.open("b.jpeg")
# im2=ImageOps.grayscale(im1)
# Arr1 = np.array(im1)
# Arr2 = np.array(im2)
# print('shape of image is',Arr.shape,' value are ',Arr[0][0])
# Arr.show()
# img = Image.fromarray(Arr)
# img.show()    

# print(x_train[0].type)
# '''
# import tensorflow as tf 
# sess = tf.Session()
# K.set_session(sess)

# class TFCheckpointCallback(keras.callbacks.Callback):
#     def __init__(self,saver,sess):
#         self.saver=saver
#         self.sess = sess
#     def on_epoch_end(self,epoch,logs=None):
#         self.saver.save(self.sess,'freeze/checkpoint',global_step=epoch)

# tf_saver = tf.train.Saver(max_to_keep=2)
# checkpoint_callback = TFCheckpointCallback(tf_saver,sess)
# sess.close()
# '''


# def prepare_graph_for_freezing(model_folder):
#     model = createModel6()
#     cp = tf.train.get_checkpoint_state(model_folder,latest_filename='cp-0020.ckpt')
#     input_cp = cp.model_checkpoint_path
#     saver = tf.train.Saver()
#     with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#         K.set_Session(sess)
#         saver.restore(sess,input_cp)
#         tf.gfile.MakeDirs(model_folder+'freeze')
#         saver.save(sess,model_folder+'freeze/checkpoint',global_step=0)

# def freeze_graph(model_folder):
#     cp = tf.train.get_checkpoint_state(model_folder,latest_filename='cp-0019.ckpt')
#     input_cp = cp.model_checkpoint_path
#     absolute_model_folder = '/'.join(input_cp.split('/')[:-1])
#     output_graph = absolute_model_folder+'/frozen_model.pb'
#     output_node_name='output/Softmax'
#     clear_devices=True
#     new_saver = tf.train.import_meta_graph(input_cp+'.meta',clear_devices=clear_devices)
#     graph=tf.get_default_graph()
#     input_graph_def = graph.as_graph_def()
    
#     with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess2:
#         new_saver.restore(sess2,input_cp)
#         output_graph_def = graph_util.convert_variables_to_constants(sess2,input_graph_def,output_node_name.split(','))
#         with tf.gfile.GFile(output_graph,'wb') as f:
#             f.write(output_graph_def.SerializeToString())

# #tf.reset_default_graph()
# #prepare_graph_for_freezing('training/Checkpoints/')
# freeze_graph('training/Checkpoints/')

In [ ]:
# print(len(x_train))
# print(len(y_train))

# print(len(x_test))
# print(len(y_test))

# a = list(zip(x_train,y_train))
# random.shuffle(a)
# x_train , y_train = zip(*a)

# a = list(zip(x_test,y_test))
# random.shuffle(a)
# x_test , y_test = zip(*a)
# del a

# x_train = np.array(list(x_train), dtype=np.float16)
# y_train = np.array(list(y_train), dtype=np.float16)
# x_test  = np.array(list(x_test), dtype=np.float16)
# y_test  = np.array(list(y_test), dtype=np.float16)
# y_train = keras.utils.np_utils.to_categorical(y_train, 20)
# y_test = keras.utils.np_utils.to_categorical(y_test, 20)

# # Normalization

# x_train /= 255;
# x_test /= 255;

# # Saving train and test set in a file

# np.save('x_train.npy',x_train)
# np.save('y_train.npy',y_train)
# np.save('x_test.npy',x_test)
# np.save('y_test.npy',y_test)

# # Loading Train and Test Set

# x_train = np.load('x_train.npy')
# y_train = np.load('y_train.npy')
# x_test  = np.load('x_test.npy')
# y_test  = np.load('y_test.npy')